# PRUEBAS ACCDB

In [ ]:
from prorrataerv.model.accdb import Querier
from prorrataerv.model.accdb.schema import QuerySchema
from prorrataerv.model.accdb.models import CategorySchema
import sqlalchemy as sa
from pathlib import Path

path_prg = Path(r"C:\Users\felipe.bastidas\PyProyectos\Test_data\PRG20241007\Datos\Model PRGdia_Full_Definitivo Solution\Model PRGdia_Full_Definitivo Solution.accdb")

connection_string = (
    r"DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};"
    rf"DBQ={path_prg.as_posix()};"
    r"ExtendedAnsiSQL=1;"
)
connection_url = sa.engine.URL.create(
    "access+pyodbc", query={"odbc_connect": connection_string}
)

eng = sa.create_engine(connection_url)

with eng.connect() as conn:
    pcp_solution = Querier(conn=conn)
    nodes = pcp_solution.get_obj_relationship(collection_id=12)
    category_list: list[CategorySchema] = pcp_solution.get_category_list(category_type=22)
    generator = pcp_solution.get_property_data(query_enum=QuerySchema.GENERATOR.GENERATION, category_list=["Solar Farms"])
    #generator_t_data = pcp_solution.get_property_t_data(query_enum=QuerySchema.GENERATOR.GENERATION)


In [2]:
from prorrataerv.business.extract import DataExtractor
from pathlib import Path

path_prg = Path(r"C:\Users\felipe.bastidas\PyProyectos\Test_data\PRG20241007\Datos\Model PRGdia_Full_Definitivo Solution\Model PRGdia_Full_Definitivo Solution.accdb")
path_banned = Path(r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Centrales_Vetadas - copia.xlsx")
path_pmgd = Path(r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Lista_PMGDs.xlsx")

extractor = DataExtractor(path_prg=path_prg, path_banned=path_banned, path_pmgd=path_pmgd)
extractor.extract_data()

In [4]:
extractor.gen

[ResultDataSchema(collection='Generators', property='Generation', category='Solar Farms', name='ABASTIBLE_CONCON_FV', datetime=datetime.datetime(2024, 10, 7, 0, 0), key_id=11187, period_id=1, value=0.0),
 ResultDataSchema(collection='Generators', property='Generation', category='Solar Farms', name='ABASTIBLE_CONCON_FV', datetime=datetime.datetime(2024, 10, 7, 1, 0), key_id=11187, period_id=2, value=0.0),
 ResultDataSchema(collection='Generators', property='Generation', category='Solar Farms', name='ABASTIBLE_CONCON_FV', datetime=datetime.datetime(2024, 10, 7, 2, 0), key_id=11187, period_id=3, value=0.0),
 ResultDataSchema(collection='Generators', property='Generation', category='Solar Farms', name='ABASTIBLE_CONCON_FV', datetime=datetime.datetime(2024, 10, 7, 3, 0), key_id=11187, period_id=4, value=0.0),
 ResultDataSchema(collection='Generators', property='Generation', category='Solar Farms', name='ABASTIBLE_CONCON_FV', datetime=datetime.datetime(2024, 10, 7, 4, 0), key_id=11187, perio

# PRUEBAS CSV y EXCEL

In [ ]:
import csv

from pydantic import BaseModel, PositiveInt, Field


class MinStableSchema(BaseModel):
    name: str = Field(alias="NAME")
    year: PositiveInt = Field(alias="YEAR")
    month: PositiveInt = Field(alias="MONTH")
    day: PositiveInt = Field(alias="DAY")
    period: PositiveInt = Field(alias="PERIOD")
    band: PositiveInt = Field(alias="BAND")
    value: float = Field(alias="VALUE")

path_minstablelevel = r"C:\Users\felipe.bastidas\PyProyectos\Test_data\PRG20241022\Datos\Gen_MinStableLevel.csv"

with open(path_minstablelevel) as file:
    reader = csv.DictReader(file)
    people = [MinStableSchema.model_validate(row) for row in reader]

print(people)

In [ ]:
import fastexcel

excel_path = r"C:\Users\felipe.bastidas\PyProyectos\Test_data\Otros\Centrales_Vetadas - copia.xlsx"

excel_reader = fastexcel.read_excel(excel_path)
data_sheet = excel_reader.load_sheet_by_idx(0, use_columns="A")
data_sheet.to_polars().to_series().to_list()